In [1]:
import bitsandbytes
import json

# with open("sql-create-context-spider-intersect-validation-with-prompts.jsonl", "r") as f:
#     data = json.load(f)

# with open('sql-create-context-spider-intersect-validation-with-prompts.jsonl', 'r') as f:
#     for line in f:
#         data = json.loads(line)
#         # print(data)

data = []
with open('/root/finetune-llm-for-rag/datasets/sql/MiniLM-L6/sql-create-context-spider-intersect-validation-with-prompts.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))
# print(data_list[2])
# Step 2: Apply the function to each entry
# responses = map(your_function, data)
def generate_prompt(data_point):
    # print("inference prompt is: ", data_point['inference_prompt'])
    return data_point['inference_prompt']
prompts = []
for data_point in data:
    # prompt = generate_prompt(data_point)
    # append prompt to prompts
    # print("INBETWEEN POUNTS:")
    prompts.append(generate_prompt(data_point))
# print(prompts)
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

base_model = "codellama/CodeLlama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
from peft import PeftModel
model = PeftModel.from_pretrained(model, "/root/finetune-llm-for-rag/training-code/fifth-rag-codellama-7b-from-checkpoint/checkpoint-580")
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

tokenizer.pad_token = tokenizer.eos_token



The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
batch_size = 16  # You can adjust the batch size based on your GPU capacity
outputs = []

model.eval()
with torch.no_grad(), open("fififth-finetune-of-rag-codellama7b.txt", "a") as f:
    for i in range(0, len(prompts), batch_size):
        print("i is: ", i)
        batch_inputs = prompts[i : i + batch_size]

        # Step 3: Loop over the batches and generate the outputs
        batch_model_inputs = tokenizer(
            batch_inputs, return_tensors="pt", padding=True, truncation=True, max_length=512
        )
        # print("length of input ids is: ", len(batch_model_inputs["input_ids"]))
        batch_model_inputs.to(model.device)
        input_lengths = [
            len(input_ids) for input_ids in batch_model_inputs["input_ids"]
        ]
        print("input lengths are: ", input_lengths)
        batch_outputs = model.generate(**batch_model_inputs, max_new_tokens=200)

        # Step 4: Collect the outputs
        batch_decoded_outputs = [
            tokenizer.decode(output[input_length:], skip_special_tokens=True)
            for output, input_length in zip(batch_outputs, input_lengths)
        ]
        outputs.extend(batch_decoded_outputs)
        # print(outputs)
        for i in range(len(batch_decoded_outputs)):
            print("input is: ", prompts[i])
            b = batch_decoded_outputs[i]
            print('full b is: ', b)
            print("B IS: ", b.split('\n')[1])
            f.write(b.split('\n')[1] + "\n")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


i is:  0
input lengths are:  [512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512]


input is:  You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question.

Given the following examples:
Example 1:
### Input:
How many artists do we have?

### Context:
CREATE TABLE artist (Id VARCHAR)

### Response:
SELECT count(*) FROM artist

Example 2:
### Input:
How many songs are there?

### Context:
CREATE TABLE Songs (Id VARCHAR)

### Response:
SELECT count(*) FROM Songs

Example 3:
### Input:
How many songs have a shared vocal?

### Context:
CREATE TABLE vocals (songid VARCHAR); CREATE TABLE songs (songid VARCHAR)

### Response:
SELECT count(DISTINCT title) FROM vocals AS T1 JOIN songs AS T2 ON T1.songid  =  T2.songid WHERE TYPE  =  "shared"

Example 4:
### Input:
How many albums are there?

### Context:
CREATE TABLE ALBUM (Id VARCHAR)

### Response:
SELECT count(*) FROM ALBUM

Example 5:
### Input:
How many songs have a lead voc

IndexError: list index out of range